In [4]:
from sportsipy.nfl.boxscore import Boxscore, BoxscorePlayer, Boxscores
from sportsipy.nfl.roster import Roster, Player, AbstractPlayer
from sportsipy.nfl.schedule import Game, Schedule
from sportsipy.nfl.teams import Team, Teams
from sportsipy.nfl.constants import *

import pandas as pd
pd.set_option('display.max_columns', None)

from pandas_profiling import ProfileReport

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# team_data = pd.read_pickle('data/df_boxscores.pkl')

# team_data.head()

In [5]:
import pickle5 as pickle
with open('data/df_boxscores.pkl', "rb") as fh:
  team_data = pickle.load(fh)


team_data.head()

,attendance,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles,away_fumbles_lost,away_interceptions,away_net_pass_yards,away_pass_attempts,away_pass_completions,away_pass_touchdowns,away_pass_yards,away_penalties,away_points,away_rush_attempts,away_rush_touchdowns,away_rush_yards,away_third_down_attempts,away_third_down_conversions,away_time_of_possession,away_times_sacked,away_total_yards,away_turnovers,away_yards_from_penalties,away_yards_lost_from_sacks,date,datetime,duration,home_first_downs,home_fourth_down_attempts,home_fourth_down_conversions,home_fumbles,home_fumbles_lost,home_interceptions,home_net_pass_yards,home_pass_attempts,home_pass_completions,home_pass_touchdowns,home_pass_yards,home_penalties,home_points,home_rush_attempts,home_rush_touchdowns,home_rush_yards,home_third_down_attempts,home_third_down_conversions,home_time_of_possession,home_times_sacked,home_total_yards,home_turnovers,home_yards_from_penalties,home_yards_lost_from_sacks,losing_abbr,losing_name,over_or_under,over_under,over_under_total,roof,stadium,surface,time,total_minus_over,total_points,vegas_line,weather,winner,winning_abbr,winning_name,won_toss,day_of_week,month,temperature,humidity,wind_chill,wind,year
200009030min,64104,23,0,0,2,1,0,272,41,27,2,290,5,27,25,1,153,15,9,32:34,2,425,1,45,18,"Sunday Sep 3, 2000",2000-09-03 13:00:00,3:08,17,0,0,2,0,1,188,23,13,0,190,4,30,30,3,186,13,6,27:26,1,374,1,20,2,CHI,Chicago Bears,over,46.5 (over),46.5,Dome,Hubert H. Humphrey Metrodome,Astroturf,1:00pm,10.5,57,Minnesota Vikings -4.5,None,Home,MIN,Minnesota Vikings,Vikings,Sunday,Sep,None,None,None,0,2000
200009030kan,78357,20,0,0,1,1,1,267,32,22,1,273,7,27,32,1,119,13,6,33:13,1,386,2,78,6,"Sunday Sep 3, 2000",2000-09-03 13:00:00,2:58,17,1,0,0,0,1,206,37,16,2,212,6,14,24,0,74,15,5,26:47,1,280,1,50,6,KAN,Kansas City Chiefs,under,44.0 (under),44.0,Outdoors,Arrowhead Stadium,Grass,1:00pm,-3.0,41,Indianapolis Colts -3.5,"90 degrees, relative humidity 50%, wind 5 mph,...",Away,CLT,Indianapolis Colts,Chiefs,Sunday,Sep,90,50%,0,5,2000
200009030was,80257,12,1,1,1,1,0,124,26,17,1,183,9,17,20,0,112,11,2,26:46,6,236,1,65,59,"Sunday Sep 3, 2000",2000-09-03 13:01:00,3:04,23,1,0,0,0,0,234,36,25,0,234,7,20,31,2,162,12,5,33:14,0,396,0,64,0,CAR,Carolina Panthers,under,47.5 (under),47.5,Outdoors,FedExField,Grass,1:01pm,-10.5,37,Washington Redskins -10.5,"80 degrees, relative humidity 98%, no wind, wi...",Home,WAS,Washington Redskins,Redskins,Sunday,Sep,80,98%,0,0,2000
200009030cle,72418,28,0,0,1,0,0,279,34,24,1,301,1,27,40,2,119,16,10,37:03,4,398,0,5,22,"Sunday Sep 3, 2000",2000-09-03 13:02:00,3:46,9,2,1,1,1,0,153,27,19,1,160,3,7,16,0,96,10,3,22:57,1,249,1,25,7,CLE,Cleveland Browns,under,38.5 (under),38.5,Outdoors,Cleveland Browns Stadium,Grass,1:02pm,-4.5,34,Jacksonville Jaguars -10.5,"78 degrees, relative humidity 63%, wind 6 mph,...",Away,JAX,Jacksonville Jaguars,Jaguars,Sunday,Sep,78,63%,0,6,2000
200009030atl,54626,23,1,0,3,0,1,247,36,23,3,253,7,28,24,1,92,9,2,28:21,1,339,1,42,6,"Sunday Sep 3, 2000",2000-09-03 13:02:00,3:13,22,0,0,2,1,0,264,31,16,2,264,4,36,32,0,95,11,4,31:39,0,359,1,51,0,SFO,San Francisco 49ers,over,46.5 (over),46.5,Dome,Georgia Dome,Astroturf,1:02pm,17.5,64,Atlanta Falcons -6.5,None,Home,ATL,Atlanta Falcons,49ers,Sunday,Sep,None,None,None,0,2000


In [6]:
print(pd.__version__)

1.2.3


## Current/Ongoing situations

### Seems like anything not played outdoors has nonetypes for the following columns:

```python
'temperature', 'humidity', 'wind_chill'
```

- Accounts for 1,404 individual games, appears to leave some other NaN values, unknown as to why for now

In [ ]:
team_data['roof'].unique()

In [ ]:
team_data[team_data['roof'].isin(['Dome', 'Retractable Roof (Open)',
                                  'Retractable Roof (Closed)'])]#['wind_chill'].unique() #'temperature', 'humidity', 
                                                                  #'wind_chill']]#.fillna(0, inplace=True)

In [ ]:
team_data[team_data['roof'] == 'Dome']

### Looking for a way to indicate whether a game was played in Prime Time (Basically after 7 PM)

In [ ]:
# Could be used to get the time of day from time column
dt = pd.to_datetime(team_data['time'], format='%I:%M%p').dt.time

### Not able to find the duration for this anywhere at the moment

- This is the only game we run into this situation, may want to just remove it

In [ ]:
# Not able to find the duration for this anywhere at the moment
team_data[team_data['duration'].isna() == True]

### Here is where I decided to figure out the columns to be used in the Multi-Indexed DataFrame

In [ ]:
team_data[['attendance', 'date', 'datetime', 'day_of_week', 'duration', 'humidity', 'losing_abbr', 
           'losing_name', 'month', 'over_or_under', 'over_under', 'over_under_total', 'roof', 
           'stadium', 'surface', 'time', 'temperature', 'total_minus_over', 'total_points', 
           'vegas_line', 'weather', 'wind','wind_chill', 'winner',
       'winning_abbr', 'winning_name', 'won_toss']].info()

# Steps to setup `get_data.py` file to get all information into DataFrame

1) Loop through Boxscores(`week=1`, `year={looped_number}`, `end_week=22`)
    - Returns Dictionary that will contain all game boxscores as strings like: `202009130min`
    - Get all these strings into a list to also loop through

[Sportsipy docs](https://sportsreference.readthedocs.io/en/stable/nfl.html#module-sportsipy.nfl.boxscore)

[Sportsipy Github](https://github.com/roclark/sportsipy/tree/ea0043747015209550abeee15df75914a58fe40b/sportsipy/nfl)

# Boxscore

* Boxscore(`Game String`)

## What was added to the created DataFrames from the API

```python
# String of full day of the week (Sunday, Monday, Thursday)
team_data['day_of_week'] = team_data['date'].str.split(expand=True)[0]
# Abbreviation of month as a string
team_data['month'] = team_data['date'].str.split(expand=True)[1]
# Temperature as integer in Farenheight
team_data['temperature'] = team_data['weather'].str.split(',', expand=True)[0].str.split(expand=True)[0]
# Relative humidity as a percentage
team_data['humidity'] = team_data['weather'].str.split(',', expand=True)[1].str.split(expand=True)[2]
# Wind chill as integer
df_boxscore['wind_chill'] = df_boxscore['weather'].str.split(',', expand=True)[3].str.split(expand=True)[2]

# Wind in MPH
df_boxscore['wind'] = df_boxscore['weather'].str.split(',', expand=True)[2]
# Converts any games played in a dome to 0, originally listed as NoneType
team_data['wind'] = team_data['wind'].apply(lambda x: 0 if x is None else x)
# Converts any games listed as 'no wind' to 0
team_data['wind'] = team_data['wind'].apply(lambda x: 0 if str(x)[-1:] == 'd' else x)
# Changing all values to the integers
team_data['wind'] = team_data['wind'].str.split(expand=True)[1].fillna(0).astype('int64')
```

# Roster

In [ ]:
game_df = pd.concat([final_df, second_df], axis=0)
game_df

In [ ]:
roster = Roster(team, year)
roster.coach, roster.players

In [ ]:
player = Player(player_id)
player

In [ ]:
player.dataframe

# Schedule

## Need to get schedules for this year, looks like it is available now

In [ ]:
schedule = Schedule('GNB', 2021)
schedule

In [ ]:
schedule.dataframe#[17]

# Team

In [ ]:
team = Team('GNB', 2021)
team

In [ ]:
team.dataframe

In [3]:
# Getting each teams season stats into Pandas DataFrame
for i in range(2000, 2021):
    
    teams = Teams(i).dataframes
        
    try:
        teams['year'] = int(i)
        final_df = pd.concat([final_df, teams], axis=0)
        print('\n', f'Gots final_df already, appending year: {i}. SMOOCHES :*', '\n')
    except:
        teams['year'] = int(i)
        final_df = teams.copy()
        print('\n', f'No final_df yet, creating from year: {i}', '\n')


 No final_df yet, creating from year: 2000 


 Gots final_df already, appending year: 2001. SMOOCHES :* 


 Gots final_df already, appending year: 2002. SMOOCHES :* 


 Gots final_df already, appending year: 2003. SMOOCHES :* 


 Gots final_df already, appending year: 2004. SMOOCHES :* 


 Gots final_df already, appending year: 2005. SMOOCHES :* 


 Gots final_df already, appending year: 2006. SMOOCHES :* 


 Gots final_df already, appending year: 2007. SMOOCHES :* 


 Gots final_df already, appending year: 2008. SMOOCHES :* 


 Gots final_df already, appending year: 2009. SMOOCHES :* 


 Gots final_df already, appending year: 2010. SMOOCHES :* 


 Gots final_df already, appending year: 2011. SMOOCHES :* 


 Gots final_df already, appending year: 2012. SMOOCHES :* 


 Gots final_df already, appending year: 2013. SMOOCHES :* 


 Gots final_df already, appending year: 2014. SMOOCHES :* 


 Gots final_df already, appending year: 2015. SMOOCHES :* 


 Gots final_df already, appending year

In [12]:
final_df

,abbreviation,defensive_simple_rating_system,first_downs,first_downs_from_penalties,fumbles,games_played,interceptions,losses,margin_of_victory,name,offensive_simple_rating_system,pass_attempts,pass_completions,pass_first_downs,pass_net_yards_per_attempt,pass_touchdowns,pass_yards,penalties,percent_drives_with_points,percent_drives_with_turnovers,plays,points_against,points_contributed_by_offense,points_difference,points_for,post_season_result,rank,rush_attempts,rush_first_downs,rush_touchdowns,rush_yards,rush_yards_per_attempt,simple_rating_system,strength_of_schedule,turnovers,win_percentage,wins,yards,yards_from_penalties,yards_per_play,year,coach
RAM,RAM,-9.5,380,21,12,16,23,6,4.3,St. Louis Rams,12.6,587,380,247,8.3,37,5232,111,46.8,16.7,1014,471,49.41,69,540,Lost WC,1,383,112,26,1843,4.8,3.1,-1.2,35,0.625,10,7075,942,7.0,2000,Mike Martz
DEN,DEN,-2.7,383,36,13,16,12,5,7.3,Denver Broncos,7.8,569,354,223,7.1,28,4243,89,38.3,11.7,1115,369,-40.21,116,485,Lost WC,2,516,124,21,2311,4.5,5.0,-2.2,25,0.688,11,6554,792,5.9,2000,Mike Shanahan
RAI,RAI,1.8,337,32,9,16,11,4,11.3,Oakland Raiders,8.0,475,284,177,6.6,28,3306,118,39.2,10.6,1023,299,11.03,180,479,Lost Conference Championship,3,520,128,23,2470,4.8,9.7,-1.5,20,0.750,12,5776,940,5.6,2000,
CLT,CLT,0.8,357,33,14,16,15,6,6.4,Indianapolis Colts,7.1,571,357,213,7.2,33,4282,89,40.4,15.2,1026,326,47.85,103,429,Lost WC,4,435,111,14,1859,4.3,7.9,1.5,29,0.625,10,6141,866,6.0,2000,
MIN,MIN,-2.3,319,19,10,16,18,5,1.6,Minnesota Vikings,4.3,495,307,193,7.2,33,3832,106,39.7,14.9,958,371,-60.12,26,397,Lost Conference Championship,5,428,107,14,2129,5.0,1.9,0.3,28,0.688,11,5961,908,6.2,2000,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DEN,DEN,-1.7,308,38,9,16,23,11,-7.7,Denver Broncos,-4.3,556,317,176,5.9,21,3451,81,33.0,17.0,1030,446,-21.82,-123,323,None,28,442,94,13,1918,4.3,-6.0,1.7,32,0.313,5,5369,670,5.2,2020,
CIN,CIN,-2.1,318,25,13,16,11,11,-7.1,Cincinnati Bengals,-5.4,581,372,201,5.5,19,3448,83,33.0,12.8,1040,424,-5.13,-113,311,None,29,411,92,13,1668,4.1,-7.5,-0.4,24,0.281,4,5116,675,4.9,2020,
JAX,JAX,-5.3,310,25,9,16,16,15,-11.6,Jacksonville Jaguars,-6.5,616,387,205,5.6,25,3699,107,30.4,14.0,997,492,-6.12,-186,306,None,30,337,80,9,1519,4.5,-11.7,-0.1,25,0.063,1,5218,1071,5.2,2020,
NYG,NYG,2.3,297,28,11,16,11,10,-4.8,New York Giants,-6.7,517,321,178,5.3,12,3026,81,33.5,12.6,966,357,-26.79,-77,280,None,31,399,91,13,1768,4.4,-4.4,0.4,22,0.375,6,4794,634,5.0,2020,


In [13]:
# Adding coach to rows
final_df['coach'] = ''

for i in range(final_df.shape[0]):
    coach = Roster(final_df['abbreviation'][i], final_df['year'][i]).coach
    print(f"Got coach {coach} from year: {final_df['year'][i]}")
    final_df['coach'][i] = coach

Got coach Mike Martz from year: 2000


/home/luke/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Got coach Mike Shanahan from year: 2000
Got coach Jon Gruden from year: 2000
Got coach Jim Mora from year: 2000
Got coach Dennis Green from year: 2000
Got coach Tony Dungy from year: 2000
Got coach Steve Mariucci from year: 2000
Got coach Tom Coughlin from year: 2000
Got coach Gunther Cunningham from year: 2000
Got coach Jim Haslett from year: 2000
Got coach Mike Sherman from year: 2000
Got coach Andy Reid from year: 2000
Got coach Jeff Fisher from year: 2000
Got coach Brian Billick from year: 2000
Got coach Jim Fassel from year: 2000
Got coach Dave Wannstedt from year: 2000
Got coach Bill Cowher from year: 2000
Got coach Al Groh from year: 2000
Got coach Mike Holmgren from year: 2000
Got coach Wade Phillips from year: 2000
Got coach George Seifert from year: 2000
Got coach Gary Moeller Bobby Ross from year: 2000
Got coach Dave Campo from year: 2000
Got coach Terry Robiskie Norv Turner from year: 2000
Got coach Bill Belichick from year: 2000
Got coach Mike Riley from year: 2000
Got coa

Got coach Joe Gibbs from year: 2006
Got coach Rod Marinelli from year: 2006
Got coach Mike McCarthy from year: 2006
Got coach Dick Jauron from year: 2006
Got coach Mike Nolan from year: 2006
Got coach Jim Mora from year: 2006
Got coach Brad Childress from year: 2006
Got coach John Fox from year: 2006
Got coach Gary Kubiak from year: 2006
Got coach Nick Saban from year: 2006
Got coach Romeo Crennel from year: 2006
Got coach Jon Gruden from year: 2006
Got coach Art Shell from year: 2006
Got coach Bill Belichick from year: 2007
Got coach Wade Phillips from year: 2007
Got coach Tony Dungy from year: 2007
Got coach Mike McCarthy from year: 2007
Got coach Norv Turner from year: 2007
Got coach Jack Del Rio from year: 2007
Got coach Ken Whisenhunt from year: 2007
Got coach Romeo Crennel from year: 2007
Got coach Mike Tomlin from year: 2007
Got coach Mike Holmgren from year: 2007
Got coach Marvin Lewis from year: 2007
Got coach Gary Kubiak from year: 2007
Got coach Sean Payton from year: 2007
G

Got coach Chip Kelly from year: 2013
Got coach Jason Garrett from year: 2013
Got coach Marvin Lewis from year: 2013
Got coach Andy Reid from year: 2013
Got coach Mike McCarthy from year: 2013
Got coach Pete Carroll from year: 2013
Got coach Sean Payton from year: 2013
Got coach Jim Harbaugh from year: 2013
Got coach Mike McCoy from year: 2013
Got coach Jim Schwartz from year: 2013
Got coach Chuck Pagano from year: 2013
Got coach Leslie Frazier from year: 2013
Got coach Bruce Arians from year: 2013
Got coach Mike Tomlin from year: 2013
Got coach Ron Rivera from year: 2013
Got coach Mike Munchak from year: 2013
Got coach Mike Smith from year: 2013
Got coach Jeff Fisher from year: 2013
Got coach Doug Marrone from year: 2013
Got coach Mike Shanahan from year: 2013
Got coach Dennis Allen from year: 2013
Got coach John Harbaugh from year: 2013
Got coach Joe Philbin from year: 2013
Got coach Rob Chudzinski from year: 2013
Got coach Tom Coughlin from year: 2013
Got coach Rex Ryan from year: 20

ConnectionError: HTTPSConnectionPool(host='www.pro-football-reference.com', port=443): Max retries exceeded with url: /players/A/AlleNa99.htm (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fddcab636d8>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [ ]:
teams = Teams(2020)
teams

In [ ]:
teams.dataframes